In [55]:
#crowd analysis project
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
import os
import shutil
import numpy as np
import sys
from os import listdir
import cv2
from os.path import isfile,join
mypath='C:\\Users\\Akshay\\datasets\\images'


In [56]:

file_names=[f for f in listdir(mypath) if isfile(join(mypath ,f ))]#storing all file names in the file_names variable
print(str(len(file_names)) +' images loaded')


352 images loaded


In [57]:
def make_dir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

In [58]:
person_count = 0
gun_count = 0
training_size = 100
test_size = 50
training_images = []
training_labels=[]
test_images = []
test_labels = []
size = 150


person_dir_train  ='C:\\Users\\Akshay\\datasets\\crowd\\train\\persons'
gun_dir_train = 'C:\\Users\\Akshay\\datasets\\crowd\\train\\guns'
person_dir_val = 'C:\\Users\\Akshay\\datasets\\crowd\\validation\\persons'
gun_dir_val = 'C:\\Users\\Akshay\\datasets\\crowd\\validation\\guns'


make_dir(person_dir_train)
make_dir(gun_dir_train)
make_dir(person_dir_val)
make_dir(gun_dir_val)

def getZeros(number):
    if(number>10 and number<100):
        return "0"
    if (number<10):
        return "00"
    else:
        return ""

In [59]:
for i , file in enumerate(file_names):
    if file_names[i][0]=='p':#file ki index position pr agr p ata hai toh if loop me jyega
        if person_count>=training_size+test_size:#equals to 150 hogya toh skip hojyega
            continue
        else:
            image=cv2.imread(mypath+'\\'+ file)
            image=cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)#150,150 size for enlarging use intercubic
            if person_count< training_size:
                training_images.append(image)#image numpy array hai jikso list me store kr rhe fir wapis uska array bna rhe
                training_labels.append(1)#labels ki list for persons it is 1 for guns it is 0
                zeros=getZeros(person_count)#getZeros person_count ko 0,1 ke format me convert krke de deta hai
                cv2.imwrite(person_dir_train + 'person' + str(zeros) + str(person_count)+ '.jpg' ,image) 
            if person_count >= training_size and person_count <= training_size + test_size:
                test_images.append(image)
                test_labels.append(1)
                zeros=getZeros(person_count - 1000)#it starts in the folder with 999 index
                cv2.imwrite(person_dir_val + 'person' + str(zeros) + str(person_count)+ '.jpg' ,image)
        person_count+=1    
    if file_names[i][0]=='g':
        if gun_count>=training_size+test_size:
            continue
        else:
            image=cv2.imread(mypath+'\\'+ file)
            image=cv2.resize(image,(size,size),interpolation=cv2.INTER_AREA)
            if gun_count < training_size:
                training_images.append(image)
                training_labels.append(0)
                zeros=getZeros(gun_count)
                cv2.imwrite(gun_dir_train + 'guns' + str(zeros) + str(gun_count)+ '.jpg' ,image)
            if gun_count>= training_size and gun_count < training_size + test_size :
                test_images.append(image) 
                test_labels.append(0)
                zeros=getZeros(gun_count -1000)
                cv2.imwrite(gun_dir_val + 'guns' + str(zeros) + str(gun_count)+ '.jpg' ,image)
        gun_count+=1#when gun count and person count reaches 150 it stops
    
    if person_count==training_size+test_size+1 and gun_count==training_size+test_size+1:
        break
print('Training and Test data Extraction complete')

Training and Test data Extraction complete


In [60]:
#lets save our dataset's to NPZ files
#using numpy savez function to store our labeled data as NPZ files 
np.savez('guns_vs_persons_training_data.npz',np.array(training_images))
np.savez('guns_vs_persons_training_labels.npz',np.array(training_labels))
np.savez('guns_vs_persons_test_data.npz',np.array(test_images))
np.savez('guns_vs_persons_test_labels.npz',np.array(test_labels))

In [61]:
#loader function
def load_data_training_and_test(datasetname):
    npzfile=np.load(datasetname+ '_training_data.npz')
    train=npzfile['arr_0']#array format
    
    npzfile=np.load(datasetname+ '_training_labels.npz')
    train_labels=npzfile['arr_0']#array format
   
    npzfile=np.load(datasetname+ '_test_data.npz')
    test=npzfile['arr_0']#array format
    
    npzfile=np.load(datasetname+ '_test_labels.npz')
    test_labels=npzfile['arr_0']#array format
    return(train,train_labels),(test,test_labels)
   

In [62]:
for i in range(1,11):
    random=np.random.randint(0,len(training_images))
    cv2.imshow('image_'+str(i),training_images[random])
    if training_labels[random]==0:
        print(str(i)+'-Gun')
    else:
        print(str(i)+'-person')
    cv2.waitKey(0)
cv2.destroyAllWindows()

1-person
2-Gun
3-person
4-person
5-Gun
6-Gun
7-person
8-Gun
9-Gun
10-person


In [78]:
batch_size=16
num_classes=10
epochs=20
(x_train,y_train),(x_test,y_test)=load_data_training_and_test('guns_vs_persons')
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
y_train=y_train.reshape(y_train.shape[0],1)
y_test=y_test.reshape(y_test.shape[0],1)

x_train/=255
x_test/=255
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(200, 150, 150, 3)
(200, 1)
(100, 150, 150, 3)
(100, 1)


In [79]:

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D
from keras.models import load_model

In [80]:

y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [81]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding='Same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding='Same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [85]:
opt=keras.optimizers.rmsprop(lr=0.0001,decay=1e-6)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary)

TypeError: compile() missing 1 required positional argument: 'optimizer'

In [84]:

history=model.fit(x_train,y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test,y_test))
#batch size here is how many nodes in hidden layer
score=model.evaluate(x_test,y_test,verbose=0)
print('Test losses',score[0])
print('Test accuracy',score[1])

Train on 200 samples, validate on 100 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor of shape [87616,512] and type float
	 [[node training_3/RMSprop/zeros_6 (defined at C:\Users\Akshay\Anaconda3\envs\PythonGPU\lib\site-packages\keras\backend\tensorflow_backend.py:702) ]]

Caused by op 'training_3/RMSprop/zeros_6', defined at:
  File "C:\Users\Akshay\Anaconda3\envs\PythonGPU\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Akshay\Anaconda3\envs\PythonGPU\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Akshay\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Akshay\Anaconda3\envs\PythonGPU\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Akshay\Anaconda3\envs\PythonGPU\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\Akshay\Anaconda3\envs\Python